## SparkR - Classification on Wine Data

In [4]:
# Load SparkR library
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))

In [5]:
# Start Spark session
sparkR.session(master="local[*]", sparkConfig=list(spark.driver.memory="2g"))

Java ref type org.apache.spark.sql.SparkSession id 1 

In [6]:
# Read data into a Spark dataframe
sdf <- read.df("winequality-white.csv", "csv", header="true", inferSchema="true") 

In [7]:
# Cache dataframe
cache(sdf)

SparkDataFrame[id:int, fixed_acidity:double, volatile_acidity:double, citric_acid:double, residual_sugar:double, chlorides:double, free_sulfur_dioxide:double, total_sulfur_dioxide:double, density:double, pH:double, sulphates:double, alcohol:double, taste:string]

In [8]:
# Examine schema
schema(sdf)

StructType
|-name = "id", type = "IntegerType", nullable = TRUE
|-name = "fixed_acidity", type = "DoubleType", nullable = TRUE
|-name = "volatile_acidity", type = "DoubleType", nullable = TRUE
|-name = "citric_acid", type = "DoubleType", nullable = TRUE
|-name = "residual_sugar", type = "DoubleType", nullable = TRUE
|-name = "chlorides", type = "DoubleType", nullable = TRUE
|-name = "free_sulfur_dioxide", type = "DoubleType", nullable = TRUE
|-name = "total_sulfur_dioxide", type = "DoubleType", nullable = TRUE
|-name = "density", type = "DoubleType", nullable = TRUE
|-name = "pH", type = "DoubleType", nullable = TRUE
|-name = "sulphates", type = "DoubleType", nullable = TRUE
|-name = "alcohol", type = "DoubleType", nullable = TRUE
|-name = "taste", type = "StringType", nullable = TRUE

In [9]:
# Split into train & test sets
seed <- 12345
train_df <- sample(sdf, withReplacement=FALSE, fraction=0.7, seed=seed)
test_df <- except (sdf, train_df)
dim(train_df)
dim(test_df)

[1] 3406   13

[1] 1492   13

In [10]:
# Train RF model
model <- spark.randomForest(train_df, taste ~ ., type="classification", numTrees=30, seed=seed)
head(summary(model))

$formula
[1] "taste ~ ."

$numFeatures
[1] 12

$features
$features[[1]]
[1] "id"

$features[[2]]
[1] "fixed_acidity"

$features[[3]]
[1] "volatile_acidity"

$features[[4]]
[1] "citric_acid"

$features[[5]]
[1] "residual_sugar"

$features[[6]]
[1] "chlorides"

$features[[7]]
[1] "free_sulfur_dioxide"

$features[[8]]
[1] "total_sulfur_dioxide"

$features[[9]]
[1] "density"

$features[[10]]
[1] "pH"

$features[[11]]
[1] "sulphates"

$features[[12]]
[1] "alcohol"


$featureImportances
[1] "(12,[0,1,2,3,4,5,6,7,8,9,10,11],[0.05348620482576279,0.027331145746754753,0.1675110741299245,0.034403143140078946,0.04948121675107753,0.1250201192570685,0.06467665652941516,0.029478222935983192,0.12118122533669537,0.0258949338926728,0.01944352818537939,0.282092529269187])"

$maxDepth
[1] 5

$numTrees
[1] 30

In [11]:
# Predict on test data
predictions <- predict(model, test_df)
prediction_df <- collect(select(predictions, "id", "prediction"))

In [12]:
# Evaluate
library(dplyr)

actual_vs_predicted <- dplyr::inner_join(as.data.frame(sdf), prediction_df, "id") %>%
    dplyr::select (id, actual=taste, predicted=prediction)
mean(actual_vs_predicted$actual == actual_vs_predicted$predicted)
table(actual_vs_predicted$actual, actual_vs_predicted$predicted)


Attaching package: 'dplyr'

The following objects are masked from 'package:SparkR':

    arrange, between, coalesce, collect, contains, count, cume_dist,
    dense_rank, desc, distinct, explain, expr, filter, first, group_by,
    intersect, lag, last, lead, mutate, n, n_distinct, ntile,
    percent_rank, rename, row_number, sample_frac, select, sql,
    summarize, union

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



[1] 0.616622

         
          average bad good
  average     443 165   73
  bad         163 342    7
  good        153  11  135

In [13]:
# Examine first few rows of actual vs predicted wine quality values
head(actual_vs_predicted)

id,actual,predicted
5,average,average
6,average,bad
8,average,bad
9,average,bad
16,good,good
17,average,bad


In [14]:
# Save model (NOTE:  path must not exist)
write.ml(model, "wine-RF-model")